In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import data as data

source_data = list(data.read_language('eng', 'fra', data.normalize_strings))

data.log('Number of sentence pairs: {}'.format(len(source_data)))
MAX_LEN = 10

ps_filtered = list(filter(data.trim_by_length(MAX_LEN), source_data))
data.log('Number of sentence pairs after filtering: {}'.format(len(ps_filtered)))

Reading Lines... 
Number of sentence pairs: 135842
Number of sentence pairs after filtering: 107638


In [37]:
from language import get_language_pairs

eng, fra = get_language_pairs('eng', 'fra', ps_filtered)

In [38]:
word_index_pairs = [(data.sentence_to_indexes(eng, p[0]), data.sentence_to_indexes(fra, p[1])) for p in ps_filtered]

In [139]:
from model import VanillaSequenceToSequence
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn

# In[]: Build the model
BATCH_SIZE = 10
LEARNING_RATE = 1e-3

seq2seq = VanillaSequenceToSequence(eng, fra, 200, n_layers=1, bidirectional=False)
seq2seq

VanillaSequenceToSequence (
  (encoder): EncoderRNN (
    (embedding): Embedding(10736, 200)
    (gru): GRU(200, 200, batch_first=True)
  )
  (decoder): DecoderRNN (
    (embedding): Embedding(17965, 200)
    (gru): GRU(200, 200, batch_first=True)
    (output_embedding): Linear (200 -> 17965)
    (softmax): Softmax ()
  )
)

In [140]:
optimizer = optim.Adam(seq2seq.parameters(), lr=LEARNING_RATE)
criterion = nn.NLLLoss()

In [143]:
# now train
for i, (inputs, target) in enumerate(data.get_batch(word_index_pairs, BATCH_SIZE)):
    hidden = seq2seq.init_hidden(len(inputs))
    optimizer.zero_grad()
    
    input_vec = Variable(torch.LongTensor(inputs), requires_grad=False)
    target_vec = Variable(torch.LongTensor(target), requires_grad=False)
        
    outputs, output_softmaxes = seq2seq(
        input_vec,
        hidden,
        target_vec,
        0
    )
    loss = 0
    for output_softmax in output_softmaxes:
        loss += criterion(output_softmax, target_vec[:, i])
        
    loss.backward()
    optimizer.step()

RuntimeError: differentiating stochastic functions requires providing a reward